# Demolitions study

- Started with [Construction permits](https://data.austintexas.gov/Building-and-Development/Issued-Construction-Permits/3syk-w9eu/data)
- Filtered to two data sets: [Full demos](https://data.austintexas.gov/Building-and-Development/demolitions-full-post2017/4d8v-cjdw) and [partial demos](https://data.austintexas.gov/Building-and-Development/demolitions-partial-post2007/8qw5-9tag), though that may be misnomer.

In [1]:
import agate

## Download is separate

Last recorded run date was: 06/27/2018

In [2]:
%%bash
curl -L -o ../data-raw/full-downloaded.csv \
https://data.austintexas.gov/resource/4d8v-cjdw.csv?\$limit=10000
curl -L -o ../data-raw/partials-downloaded.csv \
https://data.austintexas.gov/resource/8qw5-9tag.csv?\$limit=10000

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5788k    0 5788k    0     0   483k      0 --:--:--  0:00:11 --:--:--  530k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5907k    0 5907k    0     0   543k      0 --:--:--  0:00:10 --:--:--  797k


In [3]:
#imports files and merges them
specified_types = {
    'Condominium': agate.Text(),
    'CalendarYearIssued': agate.Text(),
    'DayIssued': agate.Text(),
    'IssuedInLast30Days': agate.Text(),
    'MedGasValuation': agate.Text(),
    'MedGasValuationRemodel': agate.Text(),
    'OriginalZip': agate.Text(),
    'ContractorZip': agate.Text(),
    'ApplicantZip': agate.Text(),
    'ContractorPhone': agate.Text(),
    'ApplicantPhone': agate.Text(),
}

raw_full = agate.Table.from_csv(
    '../data-raw/full-downloaded.csv',
    column_types=specified_types
)
raw_partial = agate.Table.from_csv(
    '../data-raw/partials-downloaded.csv',
    column_types=specified_types
)
raw = agate.Table.merge([raw_full, raw_partial])

In [4]:
print('raw_full length: {}'.format(len(raw_full)))
print('raw_partial length: {}'.format(len(raw_partial)))
print('raw length: {}'.format(len(raw)))

raw_full length: 7871
raw_partial length: 6314
raw length: 14185


In [5]:
# set new column for full vs partial demo

# test of the value of WorkClass, which is our key field
def set_demotype(workclass):
    if workclass == "Demolition":
        return "Full"
    else:
        return "Partial"

# add DemoType field and fill it based on function above
demotype_field = raw.compute([
      ('DemoType',
       agate.Formula(agate.Text(),
       lambda r: set_demotype(r['WorkClass']))
      )
])

In [6]:
demotype_field.pivot('DemoType').print_table()

| DemoType | Count |
| -------- | ----- |
| Full     | 7,871 |
| Partial  | 6,314 |


In [7]:
# a look at StatusCurrent before filter
print('All Status before filtering:\n')
demotype_field.pivot('StatusCurrent').print_table()

# this filters out records based on discussions with data owner
status_filter = demotype_field.where(lambda row: row['StatusCurrent'] in [
    'Active',
    'Final'
])

# status after filter
print('\nStatus after filter:\n')
status_filter.pivot('StatusCurrent').print_table(max_column_width=None)


All Status before filtering:

| StatusCurrent        | Count |
| -------------------- | ----- |
| Active               | 1,627 |
| Final                | 9,670 |
| Withdrawn            |   212 |
| Expired              | 2,048 |
| On Hold              |     2 |
| Cancelled - Contr... |     1 |
| VOID                 |   610 |
| Aborted              |     4 |
| Closed               |     2 |
| Inactive Pending ... |     9 |

Status after filter:

| StatusCurrent | Count |
| ------------- | ----- |
| Active        | 1,627 |
| Final         | 9,670 |


## Create residential and commercial subsets

## Exports

In [8]:
# sets final table after filters
demolitions = status_filter

In [9]:
# create residential and commercial subsets
residential = demolitions.where(lambda row: row['PermitClassMapped'] == "Residential")
print('\nLength of residential subset: {}'.format(len(residential)))
commercial = demolitions.where(lambda row: row['PermitClassMapped'] == "Commercial")
print('Length of commercial subset: {}'.format(len(commercial)))


Length of residential subset: 10485
Length of commercial subset: 812


In [10]:
# export final table for Tableau and other analysis
demolitions.to_csv('../data-processed/demolitions.csv')
commercial.to_csv('../data-processed/commercial.csv')
residential.to_csv('../data-processed/residential.csv')
